# Perform Enrichment

Author: Ashley Schwartz

Date: July 17, 2023

## Purpose and Background

This tutorial explains the process of conducting gene set enrichment testing, a method employed to assess predefined biologically relevant gene sets such as pathways or biological processes. The goal is to determine if these sets contain a higher number of significant genes from an experimental dataset compared to what would be expected by random chance.

Given a dataset with differential gene expression for genes with associated p-values, we can determine the gene sets (concepts) that have significantly higher significance values than expected at random. This can be done by testing against all concept ids within a concept, or a few you might be interested in. 

A large limitation to a variety of designed enrichment methodologies is the minimal information for zebrafish. To overcome this, we have developed a new organism type, 'dreM' that is a mirror of the human 'hsa' organism with zebrafish genes. For example, the KEGG database has 355 annotated pathways for humans (hsa) at the time of writing this tutorial. Zebrafish (dre) have only 177. We have mapped all genes in the 355 hsa pathways to create 355 dreM pathways.

If you would like to reference true zebrafish pathways, choose `org = 'dre'`

If you would like to reference mapped zebrafish pathways, choose `org = 'dreM'`

Some key definitions:

| Term | Definition/Description |
| - | - |
| concept | a database resource such as KEGG or GO |
| KEGG | defined as a concept https://www.kegg.jp/kegg/pathway.html | 
| KEGG pathway | pathway is a database for the KEGG concept |
| KEGG disease | disease is a database for the KEGG concept | 

In this tutorial we will be using the following key elements:

| Item | Desctiption  |
|-|-|
`data/test_data/TPP.txt` | A differential expression dataset containing Gene IDs, Log2FC values, and associated p-values |

In general, while you do not need a large foundation in Python to execute the code listed in this tutorial, a general understanding of absolute and relative paths is useful.


## Set up Python environment

In [1]:
# IMPORT PYTHON PACKAGES
# ----------------------

# makes the notebook cell print all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# path packages
import sys
from pathlib import Path
# data processing packages
import pandas as pd

In [2]:
# SET UP MY LOCAL PACKAGE
# -----------------------
# this step is only needed because the local package has not been released through pip

cwd = Path().absolute()

package_folder = cwd / Path('../src/danRerLib')
sys.path.append(str(package_folder))
import mapping, KEGG, enrichment, utils

# SET UP DATA DIRECTORY
# ---------------------
test_data_dir = cwd / Path('data/test_data/')
out_data_dir = cwd / Path('data/out_data/')

# note: I am using the Path package to take care of any operating
#       system differences for users of this tutorial

## KEGG Enrichment 

### KEGG Pathway Enrichment

_Purpose: Given a dataset with genes and associated p-values for significant differential expression, determine the enriched KEGG pathways._

__Step 1:__ Read your data into the workspace. The supported format is a dataset with columns 'NCBI Gene ID' and 'Pvalue'. We will be using the test dataset with relative path `data/test_data/TPP.txt` which is an example differential expression dataset.

In [3]:
file_path = test_data_dir / Path('TPP.txt')
tpp_df = pd.read_csv(file_path, sep='\t')

We can also quickly print some stats to see what we are working with.

In [4]:
rows, cols = tpp_df.shape
print(f"- The column names for this dataset are: {tpp_df.columns.values}")
print(f'- The data has {rows} entries (genes).')

- The column names for this dataset are: ['NCBI Gene ID' 'PValue' 'logFC']
- The data has 21854 entries (genes).


__Step 2:__ Specify the Gene ID type currently used in your dataset. 

In [5]:
id_type = 'NCBI Gene ID'

As you'l notice, this matches my first column name in my dataset. A quick reminder that the Gene ID type must be one of the supported types and is case/spelling sensitive. Options are: NCBI Gene ID, ZFIN ID, Symbol, Ensembl ID. Many of the databases use the NCBI Gene ID so that is usually a preferred format, but any ID type will work here.

__Step 3:__ Launch the `enrich_KEGG` function. This function performs enrichment for the KEGG database. I chose the 'dreM' organism so the function will perform enrichment for all mapped zebrafish pathways using the logistic method. 

_note: this can take a while since we are testing all 355 pathways_

In [6]:
out = enrichment.enrich_KEGG(tpp_df, gene_id_type = id_type, org = 'dreM')

__Step 5:__ Visualize results and print some stats. We can take a peak at the first three lines of the dataframe output and see what is returned:

In [7]:
out.head(3)

,Concept Type,Concept ID,Concept Name,# Genes in Concept in Universe,# Sig Genes Belong to Concept,Proportion of Genes,Coeff,P-value,FDR,Odds Ratio,Enriched
0,KEGG pathway,dreM04550,Signaling pathways regulating pluripotency of ...,155,47,0.303226,0.255060,4.961454e-07,4.961454e-07,1.290539,enriched
1,KEGG pathway,dreM05225,Hepatocellular carcinoma,173,56,0.323699,0.236088,2.403849e-06,2.403849e-06,1.266286,enriched
2,KEGG pathway,dreM04961,Endocrine and other factor-regulated calcium r...,64,18,0.281250,0.314233,2.836166e-06,2.836166e-06,1.369209,enriched


| Column Name | Description | Notes | 
| - | -|-|
| Concept Type | the database used | KEGG pathway is the default database and the database used here |
| Concept ID | the unique ID for the concept | KEGG pathway ids are of the form [org][5 didgit ID]. We chose the mapped zebrafish organism 'dreM' | 
| # Genes in Concept in Universe | The number of genes that are in the Concept ID that have been found in the given dataset | the number of genes in the pathway that were found in the `TPP.txt` dataset we began with |
| # Sig Genes Belong to Concept | the number of significant genes that belong to the Concept ID that are in the given dataset | the number of significant genes from our dataset that are in the KEGG pathway |
| Proportion of Genes | the proportion of significant genes in the concept ID | ... | 
| Coeff	| the $\beta$ coefficient from the logistic regression fit | ... |
| P-value | significance of enrichment | ... |
| FDR | adjusted p-value for significant of enrichment | ... |
| Odds Ratio | ... | ...| 
| Enriched | either enriched or depleted | ... |


To check how many pathways we have significantly enriched, we would check the shape of the dataframe:

In [8]:
rows, cols = out.shape
print(f'There are {rows} pathways that meet the significance criteria.')

There are 355 pathways that meet the significance criteria.


__Step 6:__ Save the results to a file. 

In [9]:
out_file_name = Path('entrich_out.txt')
out.to_csv(out_data_dir / out_file_name, sep='\t', index = False)

Other running options include:

    - method: 'logistic' (default) or 'fishers'
    - org: 'dreM' (default) or 'dre', 'hsa'
    - database: 'pathway' (default) or 'disease'
    - concept_ids: give a list of ids
    - sig_gene_cutoff_pvalue: p-value cutoff for significant genes in set
    - sig_conceptID_cutoff_pvalue: return concept ids only less than specified value for the p-value
    - sig_conceptID_cutoff_FDR: return concept ids only less than specified value for the FDR

note: if you would like to perform enrichment on KEGG disease, a list of IDs is currently required. There are many KEGG disease options at this time and have not yet been included in the database. 

For example, if you would like a stricter gene significance cutoff, you could run the function as follows:

`enrichment.enrich_KEGG(tpp_df, sig_gene_cutoff_pvalue = 0.05)`

### KEGG Pathway Enrichment Specified Pathway Set

_Purpose: Given a list of pathways, determine if the pathways are enriched in the given dataset._

__Step 1:__ Read in your data.

In [10]:
file_path = test_data_dir / Path('TPP.txt')
tpp_df = pd.read_csv(file_path, sep='\t')

__Step 2:__ Read in the Pathway IDs you are interested in, or copy/paste them directly to the shell.

In [11]:
file_path = test_data_dir / Path('kegg_pathways.txt')
pathway_ids = pd.read_csv(file_path, sep='\t')
pathway_ids

,KEGG Pathway ID
0,4911
1,4910
2,4922
3,4972
4,4930
5,4940
6,4950
7,4931


_Note:_ There are a variety of ways you can have you pathway IDs listed. If you go on the [KEGG pathway website](https://www.genome.jp/kegg/pathway.html), you'll notice that the pathways that are listed often times begin with a '0'. For example, the first pathway 'Metabolic Pathways' has ID 01100. If you were to reference this pathway for humans, it would be 'hsa01100' and 'dre01100' for zebrafish. If you would like to reference the mapped pathway as included in this library, it would be 'dreM01100'. The pathways in the file I read in just have the pathway ID. If you read this into python using pandas, python drops leading zeros unless you specify to read in the data as a string:

`pathway_ids = pd.read_csv(file_path, sep='\t', astype(str))`

If you do not take this extra step, that is okay, the program will automatically add any necessary leading zeros and add on the specified organism. 

For example, if you give an id '01100' and 'org = 'dreM', the program will reference the pathway id as 'dreM01100'.

__Step 3:__ Initialize important parameters.

In [12]:
zfish_gene_type = 'NCBI Gene ID' # the gene type in the tpp_df

__Step 4:__ Launch enrichment function.

In [13]:
out = enrichment.enrich_KEGG(tpp_df, 
                             gene_id_type = zfish_gene_type,
                             org = 'dreM',
                             database = 'pathway',
                             concept_ids = pathway_ids)

In [14]:
out

,Concept Type,Concept ID,Concept Name,# Genes in Concept in Universe,# Sig Genes Belong to Concept,Proportion of Genes,Coeff,P-value,FDR,Odds Ratio,Enriched
0,KEGG pathway,dreM04930,Type II diabetes mellitus,52,18,0.346154,0.281483,0.000423,0.000423,1.325094,enriched
1,KEGG pathway,dreM04911,Insulin secretion,105,27,0.257143,0.212893,0.001377,0.001377,1.237253,enriched
2,KEGG pathway,dreM04931,Insulin resistance,125,31,0.248000,0.160865,0.017926,0.017926,1.174526,enriched
3,KEGG pathway,dreM04910,Insulin signaling pathway,149,32,0.214765,0.143629,0.025852,0.025852,1.154456,enriched
4,KEGG pathway,dreM04922,Glucagon signaling pathway,124,29,0.233871,0.124582,0.088239,0.088239,1.132675,enriched
5,KEGG pathway,dreM04940,Type I diabetes mellitus,13,2,0.153846,-0.448269,0.323700,0.323700,0.638733,depleted
6,KEGG pathway,dreM04972,Pancreatic secretion,87,14,0.160920,0.078379,0.406909,0.406909,1.081532,enriched
7,KEGG pathway,dreM04950,Maturity onset diabetes of the young,23,4,0.173913,-0.140424,0.575148,0.575148,0.868990,depleted


As you can see, the only pathways that were tested were those included in the concept IDs given to the program. 

## Gene Ontology (GO) Enrichment

### GO Concept Enrichment

_Purpose: Given a dataset with genes and associated p-values for significant differential expression, determine the enriched GO concept ids._

__Step 1:__ Read your data into the workspace. The supported format is a dataset with columns 'NCBI Gene ID' and 'Pvalue'. We will be using the test dataset with relative path `data/test_data/TPP.txt` which is an example differential expression dataset.